## Installs
Note: If u click on images, they zoom


In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0
Fri Jan  7 12:14:56 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |            

In [ ]:
!pip install cellpose

## Imports

In [ ]:
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
!pip install opencv-python-headless==4.1.2.30
import cv2
from matplotlib.colors import from_levels_and_colors
from skimage import data, img_as_float
from skimage import exposure
from skimage.filters import threshold_otsu
import pandas as pd
from skimage import color, morphology
from scipy import ndimage

In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import skimage.io 
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
mpl.rcParams['figure.dpi'] = 300

from urllib.parse import urlparse

from cellpose import models, io, core

use_GPU = core.use_gpu()
print('>>> GPU activated? %d'%use_GPU)

creating new log file
2022-01-07 12:15:08,754 [INFO] WRITING LOG OUTPUT TO /root/.cellpose/run.log
2022-01-07 12:15:17,315 [INFO] ** TORCH CUDA version installed and working. **
>>> GPU activated? 1


In [ ]:
# import library
from google.colab import drive

#mount the drive
drive.mount('/content/drive')
# go to the url and get the password for the drive

Mounted at /content/drive


## Utils

In [ ]:
#%% SECTION de definition et d'imports a lancer au debut
# Si vous êtes dans colab mettre colaboratory ET notebook a True
# si vous êtes en local dans un notebook mettez colbaoratory a False
#      et installez bokeh vous-même
# Si vous utilisez le script sur une machine local non pas dans un notebook 
# mettez les deux à False 
notebook=True
colaboratory=True
if notebook:
    from IPython.display import Audio
    if colaboratory:
        !pip install bokeh
    from bokeh.plotting import figure, output_file, show
    from bokeh.plotting import show as showbokeh
    from bokeh.io import output_notebook
    output_notebook()
    from bokeh.colors import Color as bcolor
    from bokeh.colors.rgb import RGB


import numpy as np

import matplotlib.pyplot as plt
import scipy 
from scipy.signal import lfilter
import scipy.io
import platform
import time

from cellpose import plot
import tempfile
import os

def RGBtoYCrCb(im,ty='8bit'): 
    assert len(im.shape)==3 and im.shape[2]==3, 'image dans un mauvais format'
    if ty=='8bit':
        delta=128 #ce delta est simplement le milieu de la plage des valeurs 
                    # possibles. 
    out=np.empty(im.shape)
    Y=0.299*im[:,:,0]+0.587*im[:,:,1]+0.114*im[:,:,2]
    Cr=(im[:,:,0]-Y)*0.713+delta
    Cb=(im[:,:,2]-Y)*0.564+delta
    return (Y,Cr,Cb)



def YCrCbtoRGB(Y,Cr,Cb,ty='8bit'):
    delta=128
    out=np.empty([*Y.shape,3],dtype=np.float32)
    out[:,:,0]=Y+1.402*(Cr-delta)
    out[:,:,1]=Y-0.34414*(Cb-128)-0.71414*(Cr-128)
    out[:,:,2]=Y+1.772*(Cb-128)
    return out
    
import skimage.transform
if notebook:
  liste_couleurs_grises=[]
  for k in range(256):
    liste_couleurs_grises.append(RGB(k,k,k))
  def affiche_pour_colab(im,normalise=True,MINI=0.0, MAXI=255.0,titre=''): #special colab, ne pas regarder
    def normalise_image_pour_bokeh(X,normalise=True,MINI=0.0, MAXI=255.0,titre=''):
      Y=(X.copy())
      if len(Y.shape)==2: 
        Y=np.zeros((*Y.shape,3))
        for k in range(3):
          Y[:,:,k]=X.copy()
   
      if normalise:
        Y-=Y.min()
        Y/=Y.max()
        Y*=255
      else:
        Y-=MINI
        Y/=(MAXI-MINI)
        Y*=255

      Y=Y.astype(np.uint8)
      sortie=np.empty(Y.shape[:2],dtype=np.uint32)
      view=sortie.view(dtype=np.uint8).reshape((Y.shape[:2]+(4,)))
      for k in range(3):
        view[:,:,k]=Y[:,:,k]
      view[:,:,3]=255
      return sortie
    img=normalise_image_pour_bokeh(np.flipud(im),normalise=normalise,MINI=MINI,MAXI=MAXI)# np.flipud(np.fliplr(im)))
    p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],title=titre)
    p.x_range.range_padding = p.y_range.range_padding = 0

    # must give a vector of images
    p.image_rgba(image=[img],x=0,y=0, dw=im.shape[1], dh=im.shape[0])
    showbokeh(p)

  def affiche(im,normalise=True,MINI=0.0, MAXI=255.0,titre='',printname=False):
      affiche_pour_colab(im,normalise=normalise,MINI=MINI, MAXI=MAXI,titre=titre) 
         # sous google colab beaucoup d''options disparaissent

else:
    def affiche(im,normalise=True,MINI=0.0, MAXI=255.0,titre='',printname=False):
        """ Cette fonction fait afficher l'image EN NIVEAUX DE GRIS 
        dans gimp. Si un gimp est deja ouvert il est utilise.
        Par defaut normalise=True. Et dans ce cas l'image est normalisee 
        entre 0 et 255 avant d'être sauvegardee.
        Si normalise=False MINI et MAXI seront mis a 0 et 255 dans l'image 
        resultat
        """
        imt=np.float32(im.copy())
        if platform.system()=='Darwin': #on est sous mac
            prephrase='open -a GIMP-2.10.app ' #certainement à adapter
            endphrase=' ' 
        else: #SINON ON SUPPOSE LINUX (si vous avez un windows je ne sais comment 
                                   #faire. Si vous savez dites-moi.)
            prephrase='gimp '
            endphrase= ' &'
    
        if normalise:
            m=imt.min()
            imt=imt-m
            M=imt.max()
            if M>0:
                imt=imt/M

        else:
            imt=(imt-MINI)/(MAXI-MINI)
            imt[imt<0]=0
            imt[imt>1]=1
    
        if titre!='':
            titre='_'+titre+'_'
        titre=titre.replace(' ','')
        nomfichier=tempfile.mktemp('TPIMA'+titre+'.png')
        commande=prephrase +nomfichier+endphrase
        skio.imsave(nomfichier,imt)
        os.system(commande)
        if printname:
            print(nomfichier)


if notebook:
    def plot(*argv):
        if len(argv)==2:
            x=argv[0]
            y=argv[1]
        elif (len(argv)==1):
            x=np.arange(0,len(y))
            y=argv[0]
        else:
            raise Exception("Erreur dans les graphiques: n'arriverait \
                pas hors de google colaboratory")
    
        p=figure()
        p.line(x,y)
        showbokeh(p)
    def stem(*argv):
        if len(argv)==2:
            x=argv[0]
            y=argv[1]
        elif len(argv)==1:
            x=np.arange(0,len(y))
            y=argv[0]
        else:
            raise Exception("Erreur dans les graphiques: \
                            n'arriverait pas hors de google colaboratory")

        p = figure()#title="simple line example", x_axis_label='x',
                    #y_axis_label='y')

        # add a line renderer with legend and line thickness
        p.segment(x,0,x, y, legend_label="Temp.", line_width=2)

        showbokeh(p)
    def show():
        return
else:
    plot=plt.plot
    stem=plt.stem
    show=plt.show # force l'affichage du graphique courant

def norm(X):
    return ((abs(X)**2).sum())**0.5
def read_image(fi):
    return np.float32(skio.imread(fi))

def normalization(img):
    pixels = np.asarray(img)
    # convert from integers to floats
    pixels = pixels.astype('float32')
    # calculate global mean and standard deviation
    mean, std = pixels.mean(), pixels.std()
    # global standardization of pixels
    pixels = (pixels - mean) / np.max(std, 1e-8)
    return pixels


In [ ]:
def cells_plot(X, y_pattern_dict, ctype, image_shape, collumn_size):
  n_classes = y_pattern_dict.shape[1]
  count = 0
  limit = collumn_size*(image_shape+2)
  out = np.zeros((2,limit))
  row = np.array([])
  n = 0
  for i, x in enumerate(X):
    c = np.argmax(y_pattern_dict[i])
    if c == ctype:
      if count == 0:
        row = np.c_[x, np.zeros((image_shape,2))]
      else:
        row = np.c_[row, np.c_[x, np.zeros((image_shape,2))]]
      count += 1
      if count == collumn_size:
        n += collumn_size
        out = np.r_[out, np.zeros((2,limit))]
        out = np.r_[out, row]
        row = np.array([])
        count = 0
  n += count
  if count != 0:
    for i in range(count, collumn_size):
        row = np.c_[row, np.c_[np.zeros((image_shape,image_shape+2))]]
    out = np.r_[out, np.zeros((2,limit))]
    out = np.r_[out, row]
  w, h = out.shape
  output = np.zeros((w, h, 3))
  output[:,:,1] = out
  return output, n
  

## Tests

In [ ]:
# same image as nucleus test
img = cv2.imread("/content/drive/MyDrive/Images/Patterns/pat_11/pat_11_1.jpg")
mask = np.load("/content/drive/MyDrive/Images/Patterns/pat_11/Masks/pat_11_1.npy")

# Cytoplams segmentation 
Segmentation from nucleus and histogram equalization of cytoplasm so that it becomes detectable

In [ ]:
def get_cyto_image(img, mask, otsu):
    
    # apply equalization to nucleus and cytoplasm separetly
    cyto_img = np.copy(img)
    cyto_img[mask!=0]=0
    clahe = cv2.createCLAHE(clipLimit=10, tileGridSize=(8,8))
    cyto_img = clahe.apply(cyto_img)
    nuc = np.copy(img)
    nuc[mask==0] = 0
    clahe = cv2.createCLAHE(clipLimit=10.0, tileGridSize=(8,8))
    nuc = clahe.apply(nuc)
    nuc[mask==0] = 0
    if otsu:
        threshOtsu = threshold_otsu(cyto_img[cyto_img!=0])
        cyto_img[cyto_img < threshOtsu] = 0
        cyto_img = skimage.morphology.erosion(cyto_img)
        cyto_img = skimage.morphology.erosion(cyto_img)
    return cyto_img, nuc

def get_imgs(img_p, mask_p, plot=True, otsu=False):
    imgori = cv2.imread(img_p)
    img = np.copy(imgori)
    mask = np.load(mask_p)
    cyto_img, nuc = get_cyto_image(img[:, :, 1], mask, otsu)
    # put cytoplasm in red channel
    img[:,:,0] = np.zeros(img.shape[:2])
    img[:,:,1] = nuc
    img[:,:,2] = cyto_img
    if plot:
        fig, axs = plt.subplots(1,3)
        axs[0].imshow(img)
        axs[1].imshow(cyto_img)
        axs[2].imshow(mask)
        plt.show()
    return imgori, img

In [ ]:
def run_cytopose(img, imgori, diameter=None):
    model = models.Cellpose(gpu=use_GPU, model_type='cyto2')

    # define CHANNELS to run segementation on
    # grayscale=0, R=1, G=2, B=3
    # channels = [cytoplasm, nucleus]
    # if NUCLEUS channel does not exist, set the second channel to 0
    # channels = [0,0]
    # IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
    # channels = [0,0] # IF YOU HAVE GRAYSCALE
    # channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
    # channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

    # or if you have different types of channels in each image
    channels = [3, 2]
    # channels = [1,1]

    # if diameter is set to None, the size of the cells is estimated on a per image basis
    # you can set the average cell `diameter` in pixels yourself (recommended) 
    # diameter can be a list or a single number for all images
    if diameter==None:
      masks, flows, styles, diams = model.eval(img, diameter=145, channels=channels)    
    else:
      masks, flows, styles, diams = model.eval(img, diameter=diameter, channels=channels)
    clahe = cv2.createCLAHE(clipLimit=10, tileGridSize=(8,8))
    imgori[:,:,1] = clahe.apply(imgori[:,:,1])
    # plot results
    from cellpose import plot
    fig = plt.figure(figsize=(12,5))
    plot.show_segmentation(fig, img, masks, flows[0], channels=channels)
    plt.tight_layout()
    plt.show()

    fig = plt.figure(figsize=(12,5))
    plot.show_segmentation(fig, imgori, masks, flows[0], channels=channels)
    plt.tight_layout()
    plt.show()
    # return img_out

# Best diameter
Without otsu algorithm

In [ ]:
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')
from cellpose import plot
# define CHANNELS to run segementation on
# grayscale=0, R=1, G=2, B=3
# channels = [cytoplasm, nucleus]
# if NUCLEUS channel does not exist, set the second channel to 0
# channels = [0,0]
# IF ALL YOUR IMAGES ARE THE SAME TYPE, you can give a list with 2 elements
# channels = [0,0] # IF YOU HAVE GRAYSCALE
# channels = [2,3] # IF YOU HAVE G=cytoplasm and B=nucleus
# channels = [2,1] # IF YOU HAVE G=cytoplasm and R=nucleus

# or if you have different types of channels in each image
channels = [3, 2]
# channels = [1,1]

img_p = "/content/drive/MyDrive/Images/Patterns/pat_11/pat_11_1.jpg"
mask_p ="/content/drive/MyDrive/Images/Patterns/pat_11/Masks/pat_11_1.npy"
imgori, img = get_imgs(img_p, mask_p, False, otsu=False)
masks_out = []
for i in range(80,170, 5):
    masks, flows, styles, diams = model.eval(img, diameter=i, channels=channels)
    fig = plt.figure(figsize=(12,5))
    plot.show_segmentation(fig, img, masks, flows[0], channels=channels)
    plt.tight_layout()
    print(f'Diameter {i}')
    plt.show() 

Output hidden; open in https://colab.research.google.com to view.

# Test for all patterns


In [ ]:
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')
path = "/content/drive/MyDrive/Images/Patterns"
for pattern in glob.glob(path + '/*'):
  pattern_name = pattern[len(path)+1:]
  print(pattern_name)
  for image in sorted(glob.glob(pattern + '/*')):
    if image[-4:] == '.jpg':
      mask = sorted(glob.glob(pattern + '/Masks/*'))[0]
      imgori, img = get_imgs(image, mask, plot=False, otsu=False)
      run_cytopose(img, imgori)
      break

Output hidden; open in https://colab.research.google.com to view.

# Cyto Fibreux
Here we can see a vast improvement in the detection of Cyto Fibreux's cells, the class that propagated the largest amount of missclassification in other cells.
Because it has stopped registering the cells that were broken or too small and now adds them as cytoplams only, which is great <3

In [ ]:
img_p = "/content/drive/MyDrive/Images/Images by class/Cyto Fibreux/Cyto Fibreux_1.jpg"
mask_p ="/content/drive/MyDrive/Images/Images by class/Cyto Fibreux/Masks/Cyto Fibreux_1.npy"
imgori, img = get_imgs(img_p, mask_p, False)
run_cytopose(img, imgori)

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
img_p = "/content/drive/MyDrive/Images/Images by class/Cyto Fibreux/Cyto Fibreux_1.jpg"
mask_p ="/content/drive/MyDrive/Images/Images by class/Cyto Fibreux/Masks/Cyto Fibreux_1.npy"
imgori, img = get_imgs(img_p, mask_p, False)
run_cytopose(img, imgori)

For small images, lower cytoplasm diameter

In [ ]:
img_p = "/content/drive/MyDrive/Images/Images by class/Mouchete SSA/Mouchete SSA_1.jpg"
mask_p ="/content/drive/MyDrive/Images/Images by class/Mouchete SSA/Masks/Mouchete SSA_1.npy"
mean_diameter = pd.read_csv('/content/drive/MyDrive/Dicts/Avarage_Nucleus.csv')
pattern_name='Mouchete SSA'
d = int(mean_diameter.loc[np.where(mean_diameter.name == pattern_name), 'nucleus_diameter'])
if d>90:
  d=145
else:
  d = (145*d)/90
imgori, img = get_imgs(img_p, mask_p, False)
run_cytopose(img, imgori, d)

Output hidden; open in https://colab.research.google.com to view.

# Application to entire dataset

Now we test these parameters to the entire dataset, without otsu

In [ ]:
model = models.Cellpose(gpu=use_GPU, model_type='cyto2')
path = "/content/drive/MyDrive/Images/Images by class"
mean_diameter = pd.read_csv('/content/drive/MyDrive/Dicts/Avarage_Nucleus.csv')
for pattern in glob.glob(path + '/*'):
  pattern_name = pattern[len(path)+1:]
  print(pattern_name)
  d = int(mean_diameter.loc[np.where(mean_diameter.name == pattern_name), 'nucleus_diameter'])
  if d>90:
    d=145
  else:
    d = (145*d)/90
  for image in sorted(glob.glob(pattern + '/*')):
    if image[-4:] == '.jpg':
      mask = sorted(glob.glob(pattern + '/Masks/*'))[0]
      imgori, img = get_imgs(image, mask, plot=False, otsu=False)
      run_cytopose(img, imgori, d)
      break

Output hidden; open in https://colab.research.google.com to view.

## Conclusions

- Best diameter: 145 overall, has problems with cells that are too small or that have to much citoplasm, but shouldn't present problems, since this maps centain types of images more to blue channel and for the small cells we can lower it
- Normalization should be done before, with adaptive histogram equalization, with clipLimit of 10 for nucleus and 10 for cytoplasm for this type of image
- Otsu and erosion doesn't help for all, so should be avoided